In [2]:
# -*- coding: utf-8 -*-
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import brentq

np.random.seed(1) # random number seed

S0    = 100
r     = 0.02
T     = 1
sigma = 0.2
K     = 100
N     = 10000
days  = 252
q     = 0

In [3]:


# def CIW_barrier(S0,K,r,sigma,N,T,sig):
#     np.random.seed(1)
#     dt = T/252
#     logS_matrix = np.zeros((int(N), 253), dtype=np.float32)
#     logS_matrix[:,0] = np.log(S0)
#     logS_std_day = sigma * np.sqrt(dt)

#     # build 10000 stock paths in 252 days 
#     for day in range(1,253):
#         logS_matrix[:,day] = np.random.normal(logS_matrix[:,day-1]+(r - 0.5 * sigma**2) * dt, logS_std_day)

#     #build Stockprice Matrix
#     S_matrix = np.exp(logS_matrix)

#     # find the path including 150
#     rows_greater_than_150 = np.where(np.any(S_matrix >= 150, axis=1))[0].tolist()

#     payoff_matrix = np.exp(-r*T) * np.maximum(K - np.exp(logS_matrix[:,-1]), 0)
#     payoff_matrix[rows_greater_than_150] = 0

#     price_barrier_option = payoff_matrix.mean()
#     price_barrier_option_std = payoff_matrix.std()

#     CI_barrier_u =  (price_barrier_option + norm.ppf(1-(1-sig)/2)*price_barrier_option_std/np.sqrt(N))
#     CI_barrier_l =  (price_barrier_option - norm.ppf(1-(1-sig)/2)*price_barrier_option_std/np.sqrt(N))
#     return CI_barrier_u - CI_barrier_l
# N= 1000000
# CIW_barrier(S0=100,K=100,r=0.02,sigma=0.2,N=N,T=1,sig=0.9)

0.031901849003016736

In [ ]:
# def CIW_EU_year(S0,K,r,sigma,N,T):
    np.random.seed(1)
    logS_mean = np.log(S0) + (r - 0.5*sigma**2) * T
    logS_std = sigma * np.sqrt(T)

    # Simulate Nx1 normal RVs
    logST = np.random.normal(logS_mean, logS_std, N)

    # Payoff of PUT for each simulation, and discounted to t=0
    payoff_vector = np.exp(-r*T) * np.maximum(K - np.exp(logST), 0)

    # GET the average as the value of PUT
    Put_price = np.mean(payoff_vector)

    # Calculate the confidence interval
    sig_payoff = np.std(payoff_vector)
    CI_Put_U = (Put_price + norm.ppf(0.95)*sig_payoff/np.sqrt(N))
    CI_Put_L = (Put_price - norm.ppf(0.95)*sig_payoff/np.sqrt(N))
    width = CI_Put_U - CI_Put_L
    return width

CIW_EU_year(S0,K,r,sigma,N,T)

In [ ]:
# # def CIW_EU_year(S0,K,r,sigma,N,T):
#     np.random.seed(1)
#     logS_mean = np.log(S0) + (r - 0.5*sigma**2) * T
#     logS_std = sigma * np.sqrt(T)

#     # Simulate Nx1 normal RVs
#     logST = np.random.normal(logS_mean, logS_std, N)

#     # Payoff of PUT for each simulation, and discounted to t=0
#     payoff_vector = np.exp(-r*T) * np.maximum(K - np.exp(logST), 0)

#     # GET the average as the value of PUT
#     Put_price = np.mean(payoff_vector)

#     # Calculate the confidence interval
#     sig_payoff = np.std(payoff_vector)
#     CI_Put_U = (Put_price + norm.ppf(0.95)*sig_payoff/np.sqrt(N))
#     CI_Put_L = (Put_price - norm.ppf(0.95)*sig_payoff/np.sqrt(N))
#     width = CI_Put_U - CI_Put_L
#     return width

# CIW_EU_year(S0,K,r,sigma,N,T)

In [7]:
def minimum_N_EU_year(S0,K,r,sigma,N,T,C):
    def objective(N):
        return CIW_EU_year(S0,K,r,sigma,int(N),T) - C
    return brentq(objective,10000000,15000000)

int(minimum_N_EU_year(S0,K,r,sigma,N,T,C=0.01))

10183089

In [15]:
def CIW_AV_year(S0,K,r,sigma,N,T):
    np.random.seed(1)
    logS_mean = np.log(S0) + (r - 0.5*sigma**2) * T
    logS_std = sigma * np.sqrt(T)
    X = np.random.normal(0, logS_std, N)
    
    logS1 = logS_mean + X
    logS2 = logS_mean - X
    payoff_vector_av = 0.5 * (np.maximum(K - np.exp(logS1), 0) + np.maximum(K - np.exp(logS2), 0))

    # mean
    Put_av = (np.exp(-r*T) * np.mean(payoff_vector_av))

    #standard deviation
    sigv_av = np.std(payoff_vector_av)

    #Confidence Interval
    CI_av_U = (Put_av + norm.ppf(0.95)*sigv_av/np.sqrt(N))
    CI_av_L = (Put_av - norm.ppf(0.95)*sigv_av/np.sqrt(N))
    
    width = CI_av_U - CI_av_L
    return width

In [17]:
def minimum_N_AV_year(S0,K,r,sigma,N,T,C):
    def objective(N):
        return CIW_AV_year(S0,K,r,sigma,int(N),T) - C
    return brentq(objective,1000000,10000000)

int(minimum_N_AV_year(S0,K,r,sigma,N,T,C=0.01))

2589155